In [1]:
import pandas as pd
import ast

import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
# Load data
df = pd.read_csv('datasets/movies_dataset.csv')
# Drop unnecessary columns
# cols_to_drop = ["id", "original_language", "video", "imdb_id", "adult", "original_title", "homepage", "runtime", "spoken_languages", "status", "genres", "vote_count", "overview", "tagline"]
# Include overview and tagline columns:
cols_to_drop = ["id", "original_language", "video", "imdb_id", "adult", "original_title", "homepage", "runtime", "spoken_languages", "status", "genres", "vote_count"]
df = df.drop(cols_to_drop, axis=1)
# Drop duplicates
df.drop_duplicates(inplace=True)

C:\Users\benja\AppData\Local\Temp\ipykernel_11432\3668436297.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('datasets/movies_dataset.csv')


In [3]:
# df.shape

In [4]:
# df.info()

In [5]:
# df.describe()

### "release_date" column cleaning

In [6]:
print(f'Number of null values in "release_date" before: {len(df[df["release_date"].isnull()])}')
# Convert the column type to datetime. If there is any error coerce (convert value to null).
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
# Drop null values
df = df.dropna(subset=['release_date'])
print(f'Number of null values in "release_date" after: {len(df[df["release_date"].isnull()])}')

Number of null values in "release_date" before: 87
Number of null values in "release_date" after: 0


### 'revenue' and 'budget' columns cleaning

In [7]:
# Replace nulls in 'revenue' and 'budget' columns with 0s 
df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(0)
# Convert 'budget' and 'revenue' columns to int
df['budget'] = df['budget'].astype(int)
df['revenue'] = df['revenue'].astype(int)

In [8]:
# Convert 'popularity' column to float
df['popularity'] = df['popularity'].astype(float)

## Columns creation

In [9]:
# Create a column named "year" extracting the year from the "release_date"
# df['release_year'] = df['release_date'].dt.year
# calculate ROI and create the 'return' column
df['returns'] = df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

## "Normalizations"

### Extract "name" from "belongs_to_collection" column

In [10]:
# Convert strings to dictionaries
df["belongs_to_collection"] = df["belongs_to_collection"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Extract collection name
df["belongs_to_collection"] = df["belongs_to_collection"].apply(lambda x: x.get("name") if isinstance(x, dict) else None)

### production_companies

In [11]:
# Convert the 'production_companies' column from a string of a list of dictionaries to an actual list of dictionaries
df['production_companies'] = df['production_companies'].apply(lambda x: ast.literal_eval(x))

# Create a new DataFrame by exploding the 'production_companies' list of dictionaries into separate rows
df = df.explode('production_companies')

# Split the 'production_companies' dictionary into separate columns
df = pd.concat([df.drop(['production_companies'], axis=1), df['production_companies'].apply(pd.Series)], axis=1)

# Drop the 'id' and '0' columns (which were created from the dictionary keys)
df = df.drop(columns=['id', 0])

# Rename the 'name' column to 'production_companies'
df = df.rename(columns={'name': 'production_companies'})

### production_countries

In [12]:
# Convert the 'production_countries' column from a string of a list of dictionaries to an actual list of dictionaries
df['production_countries'] = df['production_countries'].apply(lambda x: ast.literal_eval(x))

# Create a new DataFrame by exploding the 'production_countries' list of dictionaries into separate rows
df = df.explode('production_countries')

# Split the 'production_countries' dictionary into separate columns
df = pd.concat([df.drop(['production_countries'], axis=1), df['production_countries'].apply(pd.Series)], axis=1)

# Rename the 'name' column to 'production_countries'
df = df.rename(columns={'name': 'production_countries'})

# Drop the 'iso_3166_1' and '0' columns (which were created from the dictionary keys)
df = df.drop(columns=['iso_3166_1', 0])

### Add indexes

In [13]:
df = df.reset_index(drop=True)
df = df.rename_axis('id')

### Create "description" column and drop "overview" and "tagline"

In [14]:
df['tagline'] = df['tagline'].fillna('')
df['description'] = df['overview'] + df['tagline']
df['description'] = df['description'].fillna('')
df = df.drop(['overview', 'tagline'], axis=1)

In [15]:
df.to_csv('movies_final.csv', index=True)

### Load data to PostgreSQL server

In [16]:
load_dotenv()

# Get the connection string from the environment variable
connection_string = os.environ.get("DATABASE_CONNECTION_STRING")

# Connect to the database
engine = create_engine(connection_string)

# Replace "df" with the name of your DataFrame
df.to_sql("movies", engine, if_exists="replace")

# Commit changes and close the connection
engine.dispose()

### Query DB to check

In [ ]:
load_dotenv()

# Get the connection string from the environment variable
connection_string = os.environ.get("DATABASE_CONNECTION_STRING")

# Connect to the database
engine = create_engine(connection_string)

# create a cursor object
raw_conn = engine.raw_connection()
cur = raw_conn.cursor()

# execute a query
cur.execute("SELECT * FROM movies LIMIT 5")

# retrieve the results
rows = cur.fetchall()

# print the results
for row in rows:
    print(row)

# close the cursor and the connection
cur.close()
raw_conn.close()

(0, 'Toy Story Collection', 30000000, 21.946943, '/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', datetime.datetime(1995, 10, 30, 0, 0), 373554033, 'Toy Story', 7.7, 12.4518011, 'Pixar Animation Studios', 'United States of America', "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.")
(1, None, 65000000, 17.015539, '/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg', datetime.datetime(1995, 12, 15, 0, 0), 262797249, 'Jumanji', 6.9, 4.0430346, 'TriStar Pictures', 'United States of America', "When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which prov